# HDDL Project - Generative Inpainting

Juan AYALA, Jeong Hwan KO, Aldo MELLADO AGUILAR, Alice LALOUE, Nicolas PREVOT - 5MA



Inpainting is a technique to reconstruct an image from a corrupted version of it. For example, if an image is corrupted by removing a part of it, we can try to reconstruct the image by filling the missing part in.

In the case where we can obtain a direct copy of the missing data, inpainting is a very simple task. However, in the case where we cannot obtain a direct copy of the missing data, inpainting is a much more complex task, since the missing data has to be constructed.

In this project, we will use the [Generative Inpainting](https://arxiv.org/abs/1801.07892) technique to reconstruct images from corrupted versions of them.

The main idea is to use a generative model to generate the missing data. The generative model is trained to generate the missing data from a corrupted version of the image.

Please find below the pip command to install the required packages.



In [2]:
!pip install -r requirements.txt

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.
ERROR: Invalid requirement: '_tflow_select=2.3.0=mkl' (from line 4 of requirements.txt)
Hint: = is not a valid operator. Did you mean == ?


## Download the pretrained model

In order to use the pretrained model, we need to download it from Google Drive and extract it to **\/model_logs}**. The url is available [here](https://drive.google.com/drive/folders/1uvcDgMer-4hgWlm6_G9xjvEQGP8neW15).

The database here is the CelebFaces Attributes dataset (CelebA).It is a large-scale dataset of over 200,000 celebrity images with annotations. There is a higher resolution version of the CelebA dataset, called CelebA-HQ, which contains over 10,000 images per person. The model was trained on CelebA-HQ.

There is another dataset available called Places2, which contains images of places. If you want to test this model, please download the Places2 dataset pretrained model from [here](https://drive.google.com/drive/folders/1y7Irxm3HSHGvp546hZdAZwuNmhLUVcjO).

## Import the required packages

In [3]:
import cv2
import numpy as np
import tensorflow as tf
import neuralgym as ng
import matplotlib.pyplot as plt
from inpaint_model import InpaintCAModel

Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
def generative_inpainting(image_path, mask_path):
    FLAGS = ng.Config('inpaint.yml')
    checkpoint_dir = 'model_logs'
    model = InpaintCAModel()
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path)
    assert image.shape == mask.shape
    h, w, _ = image.shape
    grid = 8
    image = image[:h//grid*grid, :w//grid*grid, :]
    mask = mask[:h//grid*grid, :w//grid*grid, :]
    image = np.expand_dims(image, 0)
    mask = np.expand_dims(mask, 0)
    input_image = np.concatenate([image, mask], axis=2)
    sess_config = tf.ConfigProto()
    sess_config.gpu_options.allow_growth = True
    with tf.Session(config=sess_config) as sess:
        input_image = tf.constant(input_image, dtype=tf.float32)
        output = model.build_server_graph(FLAGS, input_image)
        output = (output + 1.) * 127.5
        output = tf.reverse(output, [-1])
        output = tf.saturate_cast(output, tf.uint8)
        # load pretrained model
        vars_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        assign_ops = []
        for var in vars_list:       
            vname = var.name
            from_name = vname
            var_value = tf.contrib.framework.load_variable(checkpoint_dir, from_name)
            assign_ops.append(tf.assign(var, var_value))
        sess.run(assign_ops)
        result = sess.run(output)
        return result[0][:, :, ::-1]

## Testing images.

Since we have trained the model on CelebA-HQ, we need to test it on human faces to test the model.
We have decided to apply our model on the face of Franck Ribéry, a famous French football player. When he was two years old, he and his family were involed in a car crash where he suffeed facial injuries that resulted in more than 100 stitches and two long scars on his face. 

![alt text](case1_raw.png "Title")

Our goal is to reconstruct the face of Franck Ribéry by removing artificially his scars.